In [1]:
#from google.colab import drive
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt 
import seaborn as sns
import sklearn
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#import plotly.graph_objects as go
#import plotly.express as px
#from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split

In [2]:
#i didn't run this
#drive.mount('/content/drive')

In [2]:
df = pd.read_csv('final_training_testing (1).csv', low_memory=False)
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.head(5)

,Timestamp_bid,User-Agent_bid,Region ID_bids_join_impressions,City ID,Ad Exchange,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,...,13678,13776,13800,13866,13874,14273,16593,16617,16661,16706
0,20130606000104044,mozilla/4.0 (compatible; msie 8.0; windows nt ...,80,80,1,300,250,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,20130607000103558,Mozilla/4.0 (compatible; MSIE 7.0; Windows NT ...,106,107,2,728,90,0,0,162,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20130607000103600,mozilla/5.0 (windows nt 6.1) applewebkit/537.1...,94,95,1,300,250,2,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20130610000102963,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1...,275,275,2,120,600,0,0,250,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20130608000103201,mozilla/4.0 (compatible; msie 6.0; windows nt ...,80,89,1,300,250,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
df.shape

(532897, 69)

# Question for modeling
- 100,000 records per day or should we do otherwise? 
- random sampling or keep the same distribution in bids/impressions/clicks

## Train test split

In [4]:
df['Timestamp_bid'][1]

20130607000103558

In [5]:
df['date']=df['Timestamp_bid']
df['date']=df['date'].astype(str)
df['date']=df['date'].str.slice(0,8)
df['date']=df['date'].astype(int)

In [6]:
df.groupby(['date']).size()

date
20130606     95870
20130607    111326
20130608     52270
20130609    118809
20130610     56108
20130611     50949
20130612     47565
dtype: int64

In [7]:
df['exact_time']=pd.to_datetime(df['Timestamp_bid'], format='%Y%m%d%H%M%S%f')
#df['exact_time']=pd.to_datetime(df['Timestamp_bid'], format='%H')

In [51]:
train=df[df['date']<=20130610]
test=df[df['date']>20130610]
print('train:', train.shape
      , '\ntest: ', test.shape )

train: (434383, 71) 
test:  (98514, 71)


In [52]:
train.head

<bound method NDFrame.head of             Timestamp_bid                                     User-Agent_bid  \
0       20130606000104044  mozilla/4.0 (compatible; msie 8.0; windows nt ...   
1       20130607000103558  Mozilla/4.0 (compatible; MSIE 7.0; Windows NT ...   
2       20130607000103600  mozilla/5.0 (windows nt 6.1) applewebkit/537.1...   
3       20130610000102963  Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1...   
4       20130608000103201  mozilla/4.0 (compatible; msie 6.0; windows nt ...   
...                   ...                                                ...   
532892  20130609233809996  Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1...   
532893  20130609233810097  Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...   
532894  20130609233811270  Mozilla/4.0 (compatible; MSIE 7.0; Windows NT ...   
532895  20130609233811450  Mozilla/4.0 (compatible; MSIE 6.0; Windows NT ...   
532896  20130609233811463  Mozilla/4.0 (compatible; MSIE 6.0; Windows NT ...   

        R

In [12]:
#print(max(df.iloc[0:100000].Timestamp_bid))
#print(min(test.Timestamp_bid))

# Bids to impressions

## X/y
- features: data from bids
- target: impression or not

In [10]:
hasImpression = df[df['has impression']==1]

In [11]:
cityIDranking = hasImpression.groupby(['City ID']).size().sort_values(ascending=False)

In [12]:
cityIDranking[0:5].index

Int64Index([1, 219, 217, 79, 275], dtype='int64', name='City ID')

In [13]:
features1=['exact_time'
  , 'User-Agent_bid'
  , 'Region ID_bids_join_impressions'
  , 'City ID'	
  , 'Ad Exchange'
  ,	'Ad Slot Width'	
  , 'Ad Slot Height'	
  , 'Ad Slot Visibility'	
  , 'Ad Slot Format'	
  , 'Ad Slot Floor Price'	
  , 'Bidding Price'
  , 'Advertiser ID'  

  ]

target1='has impression'

In [53]:
X1_train=train[features1]
X1_test=test[features1]

y1_train=train[target1]
y1_test=test[target1]
print('X1_train:', X1_train.shape
      , '\nX1_test: ', X1_test.shape 
      , '\ny1_train:', y1_train.shape
      , '\ny1_test: ', y1_test.shape)

X1_train: (434383, 12) 
X1_test:  (98514, 12) 
y1_train: (434383,) 
y1_test:  (98514,)


produce the train set and test set with all variables, with categorical ones in dummy form

In [15]:
X1_train_all = X1_train
X1_test_all = X1_test

In [19]:
X1_train_all=X1_train_all.drop(['User-Agent_bid'],axis=1)

X1_test_all = X1_test_all.drop(['User-Agent_bid'],axis=1)

In [20]:
print (X1_train_all.shape)
print (X1_test_all.shape)

(434383, 11)
(98514, 11)


In [21]:
X1_train_all['hour'] = pd.DatetimeIndex(X1_train_all['exact_time']).hour

X1_test_all['hour'] = pd.DatetimeIndex(X1_test_all['exact_time']).hour

In [22]:
X1_train_all['timeframe_early_morning']=np.where((X1_train_all['hour']>=2) & (X1_train_all['hour']<7),1,0)
X1_train_all['timeframe_morning']=np.where((X1_train_all['hour']>=7) & (X1_train_all['hour']<12),1,0)
X1_train_all['timeframe_afternoon']=np.where((X1_train_all['hour']>=12) & (X1_train_all['hour']<18),1,0)

X1_test_all['timeframe_early_morning']=np.where((X1_test_all['hour']>=2) & (X1_test_all['hour']<7),1,0)
X1_test_all['timeframe_morning']=np.where((X1_test_all['hour']>=7) & (X1_test_all['hour']<12),1,0)
X1_test_all['timeframe_afternoon']=np.where((X1_test_all['hour']>=12) & (X1_test_all['hour']<18),1,0)



In [23]:
print (X1_train_all.shape)
print (X1_test_all.shape)

(434383, 15)
(98514, 15)


In [24]:
X1_train_all=X1_train_all.drop(['exact_time'],axis=1)

X1_test_all = X1_test_all.drop(['exact_time'],axis=1)

In [25]:
X1_train_all=X1_train_all.drop(['hour'],axis=1)

X1_test_all = X1_test_all.drop(['hour'],axis=1)

In [26]:
print (X1_train_all.shape)
print (X1_test_all.shape)

(434383, 13)
(98514, 13)


In [27]:
X1_train_all.rename(columns = {X1_train_all.columns[0]: 'Region ID'}, inplace = True)
X1_test_all.rename(columns = {X1_test_all.columns[0]: 'Region ID'}, inplace = True)

In [28]:
def addpre(allcolumns,stringA):
    newcolumn = []
    for column in allcolumns:
        
        newstring = stringA+'_'+str(column)
        
        newcolumn.append(newstring)
    return newcolumn

def renamecolumn(dataframe,newColumnName):
    dataframe.columns = newColumnName

In [29]:
dummy_AdExchange = pd.get_dummies(X1_train_all["Ad Exchange"], drop_first=True)
dummy_AdSlotFormat = pd.get_dummies(X1_train_all["Ad Slot Format"], drop_first=True)
#dummy_regionID = pd.get_dummies(X1_train_all["Region ID"], drop_first=True)
dummy_cityID = pd.get_dummies(X1_train_all["City ID"], drop_first=True)

dummy_AdExchange_test = pd.get_dummies(X1_test_all["Ad Exchange"], drop_first=True)
dummy_AdSlotFormat_test = pd.get_dummies(X1_test_all["Ad Slot Format"], drop_first=True)
#dummy_regionID_test = pd.get_dummies(X1_test_all["Region ID"], drop_first=True)
dummy_cityID_test = pd.get_dummies(X1_test_all["City ID"], drop_first=True)




In [30]:
adExchange_column_dummy = addpre(dummy_AdExchange.columns, "AdExchange")
adSlot_column_dummy = addpre(dummy_AdSlotFormat.columns, "AdSlot")
city_column_dummy = addpre(dummy_cityID.columns,"City")

adExchange_column_dummy_test = addpre(dummy_AdExchange_test.columns, "AdExchange")
adSlot_column_dummy_test = addpre(dummy_AdSlotFormat_test.columns, "AdSlot")
city_column_dummy_test = addpre(dummy_cityID_test.columns,"City")

renamecolumn(dummy_AdExchange,adExchange_column_dummy)
renamecolumn(dummy_AdSlotFormat,adSlot_column_dummy)
renamecolumn(dummy_cityID,city_column_dummy)

renamecolumn(dummy_AdExchange_test,adExchange_column_dummy_test)
renamecolumn(dummy_AdSlotFormat_test,adSlot_column_dummy_test)
renamecolumn(dummy_cityID_test,city_column_dummy_test)

In [31]:
X1_train_all_dummies = pd.concat([X1_train_all,dummy_AdExchange,dummy_AdSlotFormat,dummy_cityID],axis=1)

X1_test_all_dummies = pd.concat([X1_test_all,dummy_AdExchange_test,dummy_AdSlotFormat_test,dummy_cityID_test],axis=1)

In [32]:
X1_train_all_dummies.shape

(434383, 387)

In [33]:
y1_train.shape

(434383,)

In [34]:
X1_test_all_dummies.shape

(98514, 384)

In [35]:
X1_train_all_dummies = X1_train_all_dummies.drop(['City ID','Ad Exchange','Ad Slot Visibility'],axis=1)

Baseline model

In [16]:
y1_train[0]

1

In [17]:
y1_train_result = y1_train.to_frame()['has impression'].value_counts()

In [18]:
emperical = y1_train_result[1]/len(y1_train)

In [19]:
emperical

0.20261842659588428

In [20]:
y1_test_baselinePredict = np.repeat(emperical,len(y1_test))

In [21]:
y1_test_baselinePredict

array([0.20261843, 0.20261843, 0.20261843, ..., 0.20261843, 0.20261843,
       0.20261843])

In [22]:
sklearn.metrics.log_loss(y1_test, y1_test_baselinePredict) 

0.6977214413148615

The code that takes very very long time to run. Don't run this

In [23]:
#from sklearn.linear_model import LogisticRegression
#logreg_model=LogisticRegression(penalty='l1', solver='saga', max_iter=100000)
#logreg_model.fit(X1_train_all_dummies, y1_train)

This part is the real preprocessing to reduce the numbers of columns to 20 ish

In [54]:
X1_train=X1_train.drop(['User-Agent_bid'],axis=1)
X1_test=X1_test.drop(['User-Agent_bid'],axis=1)
print(X1_train.shape)
print(X1_test.shape)

(434383, 11)
(98514, 11)


In [55]:
X1_train['city1']=np.where(X1_train['City ID']==1,1,0)
X1_train['city219']=np.where(X1_train['City ID']==219,1,0)
X1_train['city217']=np.where(X1_train['City ID']==217,1,0)
X1_train['city79']=np.where(X1_train['City ID']==79,1,0)
X1_train['city215']=np.where(X1_train['City ID']==215,1,0)

In [56]:
X1_train=X1_train.drop('City ID',axis=1)

In [57]:
X1_test['city1']=np.where(X1_test['City ID']==1,1,0)
X1_test['city219']=np.where(X1_test['City ID']==219,1,0)
X1_test['city217']=np.where(X1_test['City ID']==217,1,0)
X1_test['city79']=np.where(X1_test['City ID']==79,1,0)
X1_test['city215']=np.where(X1_test['City ID']==215,1,0)

X1_test=X1_test.drop('City ID',axis=1)

In [58]:
X1_train['hour'] = pd.DatetimeIndex(X1_train['exact_time']).hour

X1_test['hour'] = pd.DatetimeIndex(X1_test['exact_time']).hour

In [59]:
print(X1_train.shape)
print(X1_test.shape)

(434383, 16)
(98514, 16)


In [60]:
X1_train['timeframe_early_morning']=np.where((X1_train['hour']>=2) & (X1_train['hour']<7),1,0)
X1_train['timeframe_morning']=np.where((X1_train['hour']>=7) & (X1_train['hour']<12),1,0)
X1_train['timeframe_afternoon']=np.where((X1_train['hour']>=12) & (X1_train['hour']<18),1,0)

X1_test['timeframe_early_morning']=np.where((X1_test['hour']>=2) & (X1_test['hour']<7),1,0)
X1_test['timeframe_morning']=np.where((X1_test['hour']>=7) & (X1_test['hour']<12),1,0)
X1_test['timeframe_afternoon']=np.where((X1_test['hour']>=12) & (X1_test['hour']<18),1,0)





In [61]:
X1_train=X1_train.drop(['exact_time'],axis=1)

In [62]:
X1_train=X1_train.drop(['Advertiser ID'],axis=1)

In [63]:
X1_train=X1_train.drop(['hour'],axis=1)

In [64]:
X1_train=X1_train.drop(['Region ID_bids_join_impressions'],axis=1)

In [65]:
X1_test=X1_test.drop(['exact_time'],axis=1)
X1_test=X1_test.drop(['Advertiser ID'],axis=1)
X1_test=X1_test.drop(['hour'],axis=1)
X1_test=X1_test.drop(['Region ID_bids_join_impressions'],axis=1)

In [66]:
print(X1_train.shape)
print(X1_test.shape)

(434383, 15)
(98514, 15)


In [67]:
dummy_AdExchange = pd.get_dummies(X1_train["Ad Exchange"], drop_first=True, prefix="Exchng")
dummy_AdSlotFormat = pd.get_dummies(X1_train["Ad Slot Format"], drop_first=True,prefix="Format")
dummy_AdSlotVisibility = pd.get_dummies(X1_train["Ad Slot Visibility"], drop_first=True,prefix="Visb")

dummy_AdExchange_test = pd.get_dummies(X1_test["Ad Exchange"], drop_first=True,prefix="Exchng")
dummy_AdSlotFormat_test = pd.get_dummies(X1_test["Ad Slot Format"], drop_first=True,prefix="Format")
dummy_AdSlotVisibility_test = pd.get_dummies(X1_test["Ad Slot Visibility"], drop_first=True,prefix="Visb")



In [68]:
X1_train_dummies = pd.concat([X1_train,dummy_AdExchange,dummy_AdSlotFormat,dummy_AdSlotVisibility],axis=1)

X1_test_dummies = pd.concat([X1_test,dummy_AdExchange_test,dummy_AdSlotFormat_test,dummy_AdSlotVisibility_test],axis=1)





In [69]:
print(X1_train_dummies.shape)
print(X1_test_dummies.shape)

(434383, 22)
(98514, 22)


In [70]:
X1_train_dummies_modified = X1_train_dummies.drop(["Ad Exchange","Ad Slot Format", "Ad Slot Visibility"],axis=1)

X1_test_dummies_modified = X1_test_dummies.drop(["Ad Exchange","Ad Slot Format", "Ad Slot Visibility"],axis=1)

In [71]:
print(X1_train_dummies_modified.shape)
print(X1_test_dummies_modified.shape)

(434383, 19)
(98514, 19)


In [42]:
from sklearn.linear_model import LogisticRegression
logreg_model = LogisticRegression(max_iter=100000)

logreg_model.fit(X1_train_dummies_modified, y1_train)

LogisticRegression(max_iter=100000)

In [72]:
y1_test_predicted_1=logreg_model.predict_proba(X1_test_dummies_modified)[:,1]


In [73]:
y1_test=test[target1]
sklearn.metrics.log_loss(y1_test, y1_test_predicted_1) 


0.5636113221443281

NO NEED TO RUN AFTER HERE, AS THE PERFORMANCE OF THE MODEL WITH HYPERPARAMETER TUNING IS NOT THAT SIGNIFICANTLY BETTER.

In [64]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=3, shuffle=True,random_state=0)

In [65]:
from sklearn.linear_model import LogisticRegression
C_grid = np.logspace(-2,2,8)

loglossList=[]

for c in C_grid:
    loglossList_oneC=[]
    for train_ix, test_ix in kfold.split(X1_train_dummies_modified):
        train_X, test_X = X1_train_dummies_modified.iloc[train_ix], X1_train_dummies_modified.iloc[test_ix]
        train_y, test_y = y1_train.iloc[train_ix], y1_train.iloc[test_ix]
    
        logreg_model = LogisticRegression(C=c, max_iter=100000)
        logreg_model.fit(train_X, train_y)
    
        y1_val_predicteddProb=logreg_model.predict_proba(test_X)[:,1]
        logloss = sklearn.metrics.log_loss(test_y, y1_val_predicteddProb) 
        loglossList_oneC.append(logloss)
    loglossList.append(np.mean(loglossList_oneC))

LogisticRegression(C=0.01, max_iter=100000)

LogisticRegression(C=0.01, max_iter=100000)

LogisticRegression(C=0.01, max_iter=100000)

LogisticRegression(C=0.0372759372031494, max_iter=100000)

LogisticRegression(C=0.0372759372031494, max_iter=100000)

...

LogisticRegression(C=26.826957952797247, max_iter=100000)

LogisticRegression(C=26.826957952797247, max_iter=100000)

LogisticRegression(C=100.0, max_iter=100000)

LogisticRegression(C=100.0, max_iter=100000)

LogisticRegression(C=100.0, max_iter=100000)

In [66]:
minLoss = 100
bestIdx = 0
for idx in range(len(loglossList)):
    if loglossList[idx] <= minLoss:
        minLoss = loglossList[idx]
        bestIdx = idx

In [67]:
bestC = C_grid[bestIdx]
logreg_model2 = LogisticRegression(C=bestC, max_iter=100000)
logreg_model2.fit(X1_train_dummies_modified, y1_train)

LogisticRegression(C=26.826957952797247, max_iter=100000)

In [68]:
y1_test_predicted_2=logreg_model2.predict_proba(X1_test_dummies_modified)[:,1]
y1_test=test[target1]
sklearn.metrics.log_loss(y1_test, y1_test_predicted_2) 

0.5641104892578688

In [75]:
from sklearn.linear_model import LogisticRegression
l1_ratio = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

loglossList2=[]

for l1 in l1_ratio:
    loglossList_onel1=[]
    for train_ix, test_ix in kfold.split(X1_train_dummies_modified):
        train_X, test_X = X1_train_dummies_modified.iloc[train_ix], X1_train_dummies_modified.iloc[test_ix]
        train_y, test_y = y1_train.iloc[train_ix], y1_train.iloc[test_ix]
    
        logreg_model3 = LogisticRegression(penalty='elasticnet', solver='saga', max_iter=100000,l1_ratio=l1)
        logreg_model3.fit(train_X, train_y)
    
        y1_val_predicteddProb=logreg_model.predict_proba(test_X)[:,1]
        logloss = sklearn.metrics.log_loss(test_y, y1_val_predicteddProb) 
        loglossList_onel1.append(logloss)
    loglossList2.append(np.mean(loglossList_oneC))

LogisticRegression(l1_ratio=0.1, max_iter=100000, penalty='elasticnet',
                   solver='saga')

LogisticRegression(l1_ratio=0.1, max_iter=100000, penalty='elasticnet',
                   solver='saga')

LogisticRegression(l1_ratio=0.1, max_iter=100000, penalty='elasticnet',
                   solver='saga')

LogisticRegression(l1_ratio=0.2, max_iter=100000, penalty='elasticnet',
                   solver='saga')

LogisticRegression(l1_ratio=0.2, max_iter=100000, penalty='elasticnet',
                   solver='saga')

...

LogisticRegression(l1_ratio=0.8, max_iter=100000, penalty='elasticnet',
                   solver='saga')

LogisticRegression(l1_ratio=0.8, max_iter=100000, penalty='elasticnet',
                   solver='saga')

LogisticRegression(l1_ratio=0.9, max_iter=100000, penalty='elasticnet',
                   solver='saga')

LogisticRegression(l1_ratio=0.9, max_iter=100000, penalty='elasticnet',
                   solver='saga')

LogisticRegression(l1_ratio=0.9, max_iter=100000, penalty='elasticnet',
                   solver='saga')

In [76]:
loglossList2

[0.41877313676774675,
 0.41877313676774675,
 0.41877313676774675,
 0.41877313676774675,
 0.41877313676774675,
 0.41877313676774675,
 0.41877313676774675,
 0.41877313676774675,
 0.41877313676774675]

In [77]:
logreg_model3 = LogisticRegression(penalty='elasticnet', solver='saga', max_iter=100000,l1_ratio=0.1)
logreg_model3.fit(X1_train_dummies_modified, y1_train)

LogisticRegression(l1_ratio=0.1, max_iter=100000, penalty='elasticnet',
                   solver='saga')

In [78]:
y1_test_predicted_3=logreg_model3.predict_proba(X1_test_dummies_modified)[:,1]
y1_test=test[target1]
sklearn.metrics.log_loss(y1_test, y1_test_predicted_3) 

0.5651220163500302

In [ ]:
bestC = C_grid[minIdx]


In [ ]:
bestC

In [ ]:
y1_test_predicted=logreg_model.predict_proba(X1_test_dummies_modified)[:,1]

In [ ]:
y1_test_predicted

In [ ]:
y1_test=test[target1]
sklearn.metrics.log_loss(y1_test, y1_test_predicted) 

In [ ]:
import numpy as np
np.sort(np.absolute(list(abs(logreg_model.coef_[0]))))


In [ ]:
oneData=np.array(X1_train_dummies_modified.iloc[0]).reshape(1,-1)

In [ ]:
xlist=[]
probabilityList=[]
for setPrice in range(1,2000):
    xlist.append(setPrice)
    oneData[0,4]=setPrice
    prob=logreg_model.predict_proba(oneData)[:,1]
    probabilityList.append(prob)
    

In [ ]:
import matplotlib.pyplot as plt
plt.plot(xlist,probabilityList,'o')
plt.xlabel('Paying Price')
plt.ylabel('Probablity of winning the ad')

In [ ]:
pricelist=list(range(1,2001))

y_array=np.ones((100,2000))

for position in range(0,100):
    oneData=np.array(X1_train_dummies_modified.iloc[position]).reshape(1,-1)
    probabilityList=[]
    for setPrice in range(1,2001):
        oneData[0,4]=setPrice
        prob=logreg_model.predict_proba(oneData)[:,1]
        probabilityList.append(prob)
    y_array[position,:]=probabilityList

In [ ]:
for i in range (0,100):
    rgb = np.random.rand(3,)
    plt.plot( pricelist, y_array[i,:],color=rgb,linewidth=2)

plt.xlabel('Paying Price')
plt.ylabel('Probablity of winning the ad')  


In [ ]:
logreg_model.coef_

In [ ]:
np.ones((10,2))

In [ ]:
train_ix

In [ ]:
X1_train_dummies_modified.iloc[train_ix]

In [ ]:
cv = KFold(n_splits=5, random_state=1, shuffle=True)

C_grid = np.logspace(-2,2,10)

for c in C_grid:
    logreg_model = LogisticRegression(penalty='l1', C=c, solver='saga', max_iter=100000)
    scores = cross_val_score(logreg_model, X1_train_dummies_modified, y1_train, scoring='accuracy', cv=cv, n_jobs=-1)

In [ ]:
X1_train_modified = X1_train_dummies_modified.values




In [ ]:
X1 = np.concatenate((X1_train_modified, np.ones((X1_train_modified.shape[0],1))), axis=1)

In [ ]:
X1.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg_model = LogisticRegression(max_iter=100000000,multi_class='ovr')

logreg_model.fit(X1, y1_train)

In [ ]:
y1_train_pred=logreg_model.predict(X1)
y1_train

In [ ]:
training_accuracy=logreg_model.score(X1,y1_train) # SOLUTION
print ('Training Accuracy:',training_accuracy)


#test_accuracy=logreg_model.score(X1_test,y1_test) # SOLUTION
#print ('Testing Accuracy:',test_accuracy)

In [ ]:
from collections import Counter
result=Counter(y1_train)

result


In [ ]:
len(y1_test[y1_test==0])/len(y1_test)

# Impressions to clicks (2)

## X2/y2 

In [ ]:
features2=df.drop(['has click' #target
  , 'n_clicks' #target 1:1
  , 'User Profile IDs_bid' #useless
  , 'User Profile IDs_imp' #redundant with dummification
  , 'Timestamp_bid'
  , 'Timestamp_imp'
  , 'Timestamp' #related to click
  , 'User-Agent' #related to click
  , 'User Profile IDs' #related to click
  , 'Region ID_clicks' #related to click
  , 'Log Type' #useless in our data format 
  , 'n_impressions'

  , 'User-Agent_bid' # for now let's ignore this as string too complex
  , 'User-Agent_imp' # for now let's ignore this as string too complex
  ], axis=1).columns
target2='has click'

In [ ]:
#df[features2].isna().sum() --> we can fill na with 0 as there are some only for paying price and null profile ids

X2_train = train[features2].fillna(0)
X2_test = test[features2].fillna(0)

y2_train = train[target2]
y2_test = test[target2]

print('X2_train:', X2_train.shape
      , '\nX2_test: ', X2_test.shape 
      , '\ny2_train:', y2_train.shape
      , '\ny2_test: ', y2_test.shape)

In [ ]:
y2_train.sum()

In [ ]:
1000/10000000*700000